In [3]:
import numpy as np
import pandas as pd
from utils import physics
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt

filename = "LLNL_160809_freq.csv"
df = pd.read_csv("data/"+filename, header = None)
df.columns = ["Frequency (in THz)", "Wavelength (in nm)", "Intensity", "Phase (rad)", "Phase (cutted) (rad)"] 

frequency = df.loc[:, "Frequency (in THz)"].values * 10**12 # THz to Hz
field = np.sqrt(df.loc[:, "Intensity"].values)
central_carrier = 2294295618320813.0

num_points = int(1e3)
start_freq, end_freq = 351, 379

field_interpolator = interp1d(frequency, field)
frequency_spaced = np.linspace(start = start_freq, stop = end_freq, num = num_points, endpoint = True) * 10**12
field_spaced = field_interpolator(frequency_spaced)

frequency = frequency_spaced; field = field_spaced

ps = physics.PulseEmitter(frequency = frequency, field = field)

theorical_phase = physics.theorical_phase(frequency, central_carrier)
control_params = physics.phase_expansions(frequency, theorical_phase)
pulse = ps.temporal_profile(control_params)
timescale = ps.time_scale()

%matplotlib
fig, ax = plt.subplots()
ax.plot(timescale, pulse, label = "Initial value")


Using matplotlib backend: MacOSX


In [5]:
control_params

array([-6.96834538e+03,  9.32894922e-11, -3.54179759e-25,  4.13417022e-40,
        8.20699224e-65])

In [8]:
nrows = 4
fig, ax = plt.subplots(nrows = nrows)

for i in range(nrows): 
    current = ax[i]
    modified_control_params = control_params + 5*np.random.random(size=1) * control_params
    modified_pulse = ps.temporal_profile(modified_control_params)

    current.plot(timescale, modified_pulse)

In [ ]:
import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_model
from botorch.utils import standardize
from gpytorch.mlls import ExactMarginalLogLikelihood